# Problem

**Given**: A collection of up to 1000reads of equal length (at most 50bp) in FASTA format. Some of these reads were generated with single-nt error. For each read $s$ in the dataset, one of the following applies.

* $s$ was correctly sequenced and appears in the datasets at least twice (possibly as a reverse complement)
* $s$ is incorrect, it appears in the dataset exactly once, and its Hamming distance is 1 with respect to exactly one correct read in the dataset(or its reverse complement)

**Return**: A list of all corrections in the form "\[old-read\]->\[new-read\]". (Each correction must be single symbol substitution, and you may return the corrections in any order

In [25]:
import toolz as tz
from toolz.curried import *
from operator import *
import pandas as pd
import numpy as np

In [81]:
input_data = {
    "sample": {
        "fasta": """>Rosalind_52
TCATC
>Rosalind_44
TTCAT
>Rosalind_68
TCATC
>Rosalind_28
TGAAA
>Rosalind_95
GAGGA
>Rosalind_66
TTTCA
>Rosalind_33
ATCAA
>Rosalind_21
TTGAT
>Rosalind_18
TTTCC"""
    },
    "test": {
        "fasta": open("data/rosalind_corr.txt", "r").read()
    }
}

cur_state = "test"
cur_data = input_data[cur_state]

replace = {
    "A": "T",
    "T": "A",
    "C": "G",
    "G": "C"
}

input_processor = compose(list, filter(lambda x: not x.startswith(">")), flip(str.split,  "\n"), str.strip)

hamming_distance = lambda a, b: reduce(lambda init, cur: init + int(ne(*cur)), zip(a,b), 0)

rev_seq = lambda x: ''.join(reversed([replace[nt] for nt in x]))

def find_correct(seqs):
    correct = set()
    for i, seq_i in enumerate(seqs):
        for seq_j in seqs[:i] + seqs[i+1: ]:
            if seq_i == seq_j:
                correct = correct.union({seq_i, seq_j})
            elif seq_i == rev_seq(seq_j):
                correct = correct.union({seq_i, rev_seq(seq_j), rev_seq(seq_i), seq_j})
    return correct, seqs

def find_error(datapacakge):
    correct, seqs = datapacakge
    correct = set(tz.concat([[x, rev_seq(x)] for x in correct]))
    errors = set(seqs).difference(correct)
    return correct, errors

def correction(datapacakge):
    correct, errors = datapacakge
    pairs = []
    for error in errors:
        cor = [cor for cor in correct if hamming_distance(cor, error) == 1]
        if len(cor) == 1:
            if hamming_distance(error, first(cor)) == 1:
                pairs.append((error, first(cor)))
    return pairs

def print_pair(pairs):
    for error, correct in pairs:
        print("{}->{}".format(error, correct))

run = compose(print_pair, correction, find_error, find_correct, input_processor)

run(cur_data["fasta"])

...


In [83]:
test_output = open("test/corr_output.txt", "r").read()
output_df = pd.DataFrame(tz.pipe(test_output, flip(str.split, "\n"), map(flip(str.split, "->"))), columns=["error","correct"])

# hamming distance가 모두 1인지
output_df.groupby("correct").size()



correct
AAAACCAGGAGTACCTATATCCTCCAAATCTACAGACAGCAGACACAGAT    1
AAACCAGGAGTACCTATATCCTCCAAATCTACAGACAGCAGACACAGATG    3
AACCAGGAGTACCTATATCCTCCAAATCTACAGACAGCAGACACAGATGG    5
AAGATTGAATATACTCTAATGGGGATTGGCCCATTGAAAACCAGGAGTAC    4
AAGGTTGCTCCGATTTCTTCGGGGGATTATTCCACTGCCAAGATTGAATA    2
                                                     ..
TTGAAAACCAGGAGTACCTATATCCTCCAAATCTACAGACAGCAGACACA    2
TTGAATATACTCTAATGGGGATTGGCCCATTGAAAACCAGGAGTACCTAT    5
TTGCTCCGATTTCTTCGGGGGATTATTCCACTGCCAAGATTGAATATACT    5
TTGGCCCATTGAAAACCAGGAGTACCTATATCCTCCAAATCTACAGACAG    1
TTTCTTCGGGGGATTATTCCACTGCCAAGATTGAATATACTCTAATGGGG    6
Length: 94, dtype: int64